In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# input data : 224x224 size
img = torch.rand((1,3,224,224))

In [32]:
# 2,4개 block 단위로 묶었음 , 다른 사람들은 list안에 넣고 조금 더 효율적인듯
class VGG19(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg = nn.Sequential(
            # 처음 2회 2개 묶은 것
            self.two_block(3,64,kernel_size=3,stride=1,padding=1),
            self.two_block(64,128,kernel_size=3,stride=1,padding=1),
            # 이후 4개 묶은 것 3회
            self.four_block(128,256,kernel_size=3,stride=1,padding=1),
            self.four_block(256,512,kernel_size=3,stride=1,padding=1),
            self.four_block(512,512,kernel_size=3,stride=1,padding=1),
            # 여기까지 진행하면 512 channels, 7x7 size
            # 이후 그냥 flat? global average pooling?
            #global average pooling :  nn.AdaptiveAvgPool2d(1) -> 512x1x1
        )
        self.fc = nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.Linear(4096,4096),
            nn.Linear(4096,1000)
        )
        
    # 2개를 하나의 블록으로 묶어서 진행 후 Maxpool -> 2회
    def two_block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size= kernel_size,padding = padding,stride = stride),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(out_channels,out_channels,kernel_size=kernel_size,padding=padding,stride=stride),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.MaxPool2d(2,2)
        )
    # 2개 블록으로 묶은 것 2회 이후 4개씩 묶은 것 3회 진행
    # 진행하는 동안 channel은 바뀌지 않음 -> out_channels -> out_channels
    def four_block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size= kernel_size,padding = padding,stride = stride),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(out_channels,out_channels,kernel_size=kernel_size,padding=padding,stride=stride),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(out_channels,out_channels,kernel_size=kernel_size,padding=padding,stride=stride),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(out_channels,out_channels,kernel_size=kernel_size,padding=padding,stride=stride),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.MaxPool2d(2,2)
            )
        
    def forward(self,x):
        x = self.vgg(x)
        # flatten
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        
        return x

In [33]:
model = VGG19()
output = model(img)
print(output.shape)

torch.Size([1, 1000])


In [35]:
vgg19_list = [64,64,"M",128,128,"M",256,256,256,256,"M",512,512,512,512,"M",512,512,512,512,"M"]

In [36]:
class VGG19_other_code(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_channels = 3
        self.vgg = self.create_layer(vgg19_list)
        self.fc = nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.Linear(4096,4096),
            nn.Linear(4096,1000)
        )
        
    def create_layer(self,vgg_list):
        layers = []
        in_channels = self.in_channels
        for x in vgg_list:
            if type(x) == int:
                # 먼저 out_channel을 변경한 후에 기존의 in_channel과 conv2d 진행하기에 오류 x
                out_channels = x
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU(),
                           nn.Dropout(0.5)]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
                
        return nn.Sequential(*layers)
    
    def forward(self,x):
        x = self.vgg(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

In [37]:
model = VGG19_other_code()
output = model(img)
print(output.shape)

torch.Size([1, 1000])
